In [1]:
# Import library and some pre-installed modules
import os
import sys
import time
import json
import requests
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import glob
import threading
import csv
import math
import ipywidgets as widgets
from IPython.display import display, Markdown
from shapely.geometry import shape, box, mapping
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
from tqdm import tqdm
from datetime import datetime, timedelta
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from copy import deepcopy
from sqlalchemy import create_engine

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')
# Import the custom module

In [3]:
# Get current working directory
os.getcwd()


'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

### Import the modules

In [4]:
# Import the custom module
from src import utils

In [5]:
# Reload the utils module to ensure any changes are reflected
import importlib
importlib.reload(utils)

<module 'src.utils' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/src/utils.py'>

### **OHSOME API**

 - Access to features, attributes and OSM history edits using the OHSOME API (*OpenStreetMap History Data Analytics Platform*)

> - https://docs.ohsome.org/ohsome-api/v1/

In [6]:
# URL of OHSOME API Metadata endpoint
URL = 'https://api.ohsome.org/v1/metadata'

# Request to the OHSOME API
response = requests.get(URL)

response_json = response.json()
response_json

{'attribution': {'url': 'https://ohsome.org/copyrights',
  'text': '© OpenStreetMap contributors'},
 'apiVersion': '1.10.4',
 'timeout': 600.0,
 'extractRegion': {'spatialExtent': {'type': 'Polygon',
   'coordinates': [[[-180.0, -90.0],
     [180.0, -90.0],
     [180.0, 90.0],
     [-180.0, 90.0],
     [-180.0, -90.0]]]},
  'temporalExtent': {'fromTimestamp': '2007-10-08T00:00:00Z',
   'toTimestamp': '2025-04-06T13:00Z'},
  'replicationSequenceNumber': 110142}}

### Retrieving data from OpenStreetMap using OHSOME API

#### Define ET-EDGV class dictionary with respective OSM tags

In [7]:
# Novo dicionário de classes ET-EDGV com respectivas tags OSM
classe_et_edgv_to_tags = {
    'edif_ensino': [
        ('amenity', 'school'), ('amenity', 'university'),
        ('building', 'school'), ('amenity', 'kindergarten')
    ],
    'edif_saude': [
        ('amenity', 'hospital'), ('amenity', 'clinic'),
        ('building', 'hospital'), ('amenity', 'doctors'),
        ('amenity', 'dentist'), ('healthcare', '*')
    ],
    'edif_desenv_social': [
        ('amenity', 'social_facility'), ('building', 'public'),
        ('social_facility', '*')
    ],
    'edif_constr_lazer': [
        ('leisure', 'park'), ('leisure', 'sports_centre'),
        ('leisure', 'stadium'), ('amenity', 'theatre'),
        ('amenity', 'library'), ('amenity', 'community_centre'),
        ('amenity', 'arts_centre'), ('amenity', 'planetarium'),
        ('building', 'grandstand'), ('building', 'stadium'),
        ('tourism', 'museum')
    ],
    'edif_pub_civil': [
        ('building', 'public'), ('amenity', 'townhall'),
        ('office', 'government')
    ],
    'edif_turistica': [
        ('tourism', 'attraction'), ('tourism', 'artwork'),
        ('tourism', 'viewpoint'), ('amenity', 'fountain'),
        ('building', 'hotel')
    ],
    'edif_metro_ferroviaria': [
        ('railway', 'station'), ('railway', 'halt'),
        ('building', 'train_station'), ('public_transport', 'station')
    ]
}

#### Retrieval of the last toponyms by the most recent feature contribution for each grid cell with name_ratio > 0

1. Conectar ao PostGIS para filtrar apenas células onde name_ratio > 0 para pelo menos uma classe.

2. Pré-filtragem das por células das classe com as tags do dicionário classe_et_edgv_to_tags - aumentar performance.

3. Para cada célula e classe com name_ratio > 0:

  * Extrair bbox da célula válida;

  * Uso do inflexao_data (step6) como início da janela temporal

  * Fazer chamada A API OSHOME para para recuperar a contribuição mais recente com name=*
    - Endpoint: POST /contributions/latest/geometry;

  * Resgatar geometria (pontos) e metadados:
    - Metadados: @timestamp, @osmId, tags, name.

4. Salvar os resultados em GeoJSON incremental.

##### PostGIS - Open the database connection

In [8]:
# Conexão ao Banco PostGIS
import psycopg2

# Function to load database credentials from a text file
def load_credentials_from_txt(file_path):
    credentials = {}
    try:
        with open(file_path, 'r') as f:
            for line in f:
                if '=' in line:
                    key, value = line.strip().split('=', 1)
                    credentials[key.strip()] = value.strip()
    except FileNotFoundError:
        print(f"Arquivo de credenciais não encontrado: {file_path}")
    except Exception as e:
        print(f"Erro ao ler credenciais: {e}")
    return credentials

def connect_to_postgis(txt_path='configs/db_credentials.txt'):
    creds = load_credentials_from_txt(txt_path)

    required_keys = ['DB_NAME', 'DB_USER', 'DB_PASSWORD', 'DB_HOST', 'DB_PORT']
    if not all(k in creds for k in required_keys):
        print("Credenciais incompletas no arquivo de configuração.")
        return None

    try:
        conn = psycopg2.connect(
            dbname=creds['DB_NAME'],
            user=creds['DB_USER'],
            password=creds['DB_PASSWORD'],
            host=creds['DB_HOST'],
            port=creds['DB_PORT']
        )
        print("Conexão ao PostGIS estabelecida com sucesso!")
        return conn

    except Exception as e:
        print(f" Erro ao conectar ao PostGIS: {e}")
        return None

In [9]:
# Open the database connection
conn = connect_to_postgis()

Conexão ao PostGIS estabelecida com sucesso!


##### Filter grid cells from the database with name_ratio > 0

 * Pré-filtragem SQL
 * Garantir que pelo menos uma classe tem name_ratio > 0 para cada célula da grade


In [12]:
# Function to fetch valid grid cells from the database
from shapely import wkt

def fetch_geometries_psycopg2(conn, classe_et_edgv_to_tags, table_name):
    """
    Recupera geometrias do PostGIS com base nos filtros name_ratio > 0 usando psycopg2.
    Retorna um GeoDataFrame.
    """
    try:
        # Garante que qualquer erro anterior seja limpo
        conn.rollback()

        # Monta cláusula WHERE
        where_clause = " OR ".join([
            f"step1_consolidado_{classe}_name_ratio > 0" for classe in classe_et_edgv_to_tags
        ])

        query = f"""
            SELECT *, ST_AsText(geom) AS wkt_geom
            FROM public.{table_name}
            WHERE {where_clause}
        """

        with conn.cursor() as cur:
            cur.execute(query)
            colnames = [desc[0] for desc in cur.description]
            rows = cur.fetchall()

        # Monta DataFrame
        df = pd.DataFrame(rows, columns=colnames)

        # Converte geometria WKT em shapely
        df["geometry"] = df["wkt_geom"].apply(wkt.loads)
        gdf = gpd.GeoDataFrame(df.drop(columns=["wkt_geom"]), geometry="geometry")

        # Define CRS padrão
        gdf.set_crs(epsg=4674, inplace=True)

        print(f"Consulta retornou {len(gdf)} registros.")
        return gdf

    except psycopg2.Error as e:
        conn.rollback()
        print("Erro ao executar a consulta SQL:")
        print(e.pgerror)
        raise

In [13]:
# Get the valid grid cell from the database using psycopg2
gdf_cells_valid = fetch_geometries_psycopg2(conn, classe_et_edgv_to_tags, table_name="steps_consolidado_20cells_tests")
display(gdf_cells_valid.head())

Consulta retornou 20 registros.


,id,geom,fid,POP10,step1_consolidado_fid,step1_consolidado_edif_ensino_total_count,step1_consolidado_edif_ensino_name_count,step1_consolidado_edif_ensino_name_ratio,step1_consolidado_edif_saude_total_count,step1_consolidado_edif_saude_name_count,...,step6_consolidado_edif_metro_ferroviaria_sigmoid_pct_erro,step6_consolidado_edif_metro_ferroviaria_sigmoid_a,step6_consolidado_edif_metro_ferroviaria_sigmoid_b,step6_consolidado_edif_metro_ferroviaria_sigmoid_c,step6_consolidado_edif_metro_ferroviaria_sigmoid_d,step6_consolidado_edif_metro_ferroviaria_inflexao_idx,step6_consolidado_edif_metro_ferroviaria_inflexao_data,step6_consolidado_edif_metro_ferroviaria_sigmoid_fit_overflow,step6_consolidado_edif_metro_ferroviaria_dias_desde_inflexao,geometry
0,200ME60430N90998,0106000020421200000100000001030000000100000005...,1,118,1966,1.0,1.0,100.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-43.97936 -19.87093, -43.97942..."
1,200ME60432N90998,0106000020421200000100000001030000000100000005...,2,30,1967,2.0,2.0,100.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-43.97744 -19.87086, -43.97750..."
2,200ME60434N90998,0106000020421200000100000001030000000100000005...,3,0,1968,1.0,1.0,100.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-43.97552 -19.87080, -43.97558..."
3,200ME60436N90998,0106000020421200000100000001030000000100000005...,4,0,1969,2.0,2.0,100.0,1.0,1.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-43.97360 -19.87073, -43.97367..."
4,200ME60438N90998,0106000020421200000100000001030000000100000005...,5,0,1970,2.0,2.0,100.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-43.97168 -19.87067, -43.97175..."


In [14]:
gdf_cells_valid.columns.tolist()  # para garantir que os nomes estão corretos

['id',
 'geom',
 'fid',
 'POP10',
 'step1_consolidado_fid',
 'step1_consolidado_edif_ensino_total_count',
 'step1_consolidado_edif_ensino_name_count',
 'step1_consolidado_edif_ensino_name_ratio',
 'step1_consolidado_edif_saude_total_count',
 'step1_consolidado_edif_saude_name_count',
 'step1_consolidado_edif_saude_name_ratio',
 'step1_consolidado_edif_desenv_social_total_count',
 'step1_consolidado_edif_desenv_social_name_count',
 'step1_consolidado_edif_desenv_social_name_ratio',
 'step1_consolidado_edif_constr_lazer_total_count',
 'step1_consolidado_edif_constr_lazer_name_count',
 'step1_consolidado_edif_constr_lazer_name_ratio',
 'step1_consolidado_edif_pub_civil_total_count',
 'step1_consolidado_edif_pub_civil_name_count',
 'step1_consolidado_edif_pub_civil_name_ratio',
 'step1_consolidado_edif_turistica_total_count',
 'step1_consolidado_edif_turistica_name_count',
 'step1_consolidado_edif_turistica_name_ratio',
 'step1_consolidado_edif_metro_ferroviaria_total_count',
 'step1_conso

##### Step 7 – Últimos Topônimos

Consulta à API contributions/latest/geometry
Filtro por célula e classe com name_ratio > 0
Respeito à data de inflexão (step6_consolidado_{classe}_inflexao_data)
Armazenamento de resultados como pontos (centroides se necessário)
Paralelização por célula
Salvamento incremental em GeoJSON por lote
Log detalhado e consolidação final

In [15]:
# === STEP 7: Último topônimo OSM por classe ===
# Versão final com paralelização, logs, output GeoJSON e metadados completos

# === CONFIGURAÇÕES GERAIS ===
output_dir = Path("data/output_code1/20cells_tests/step7_latest_name")
output_dir.mkdir(parents=True, exist_ok=True)

log_path = output_dir / "log_step7.csv"
ultimo_lote_path = output_dir / "ultimo_lote_step7.txt"
url_ohsome_latest = "https://api.ohsome.org/v1/contributions/latest/geometry"

# === INICIALIZAÇÃO DO LOG ===
if not log_path.exists():
    with open(log_path, 'w', newline='') as f:
        csv.writer(f).writerow(["lote", "mensagem", "timestamp"])

def log_mensagem(lote, mensagem):
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    with open(log_path, 'a', newline='') as f:
        csv.writer(f).writerow([lote, mensagem, timestamp])

# === KEEP ALIVE (com controle de término) ===
# Flag global para controle
keep_alive_running = True

def keep_alive():
    while keep_alive_running:
        time.sleep(300)
        print("Ainda trabalhando...")
        log_mensagem("keep_alive", "Ainda trabalhando...")

keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

# === FUNÇÃO PARA PROCESSAR UMA CÉLULA ===
def processar_ultima_contribuicao(cell_row):
    id_celula = cell_row["id"]
    bbox = cell_row.geometry.bounds  # (minx, miny, maxx, maxy)
    features_resultantes = []

    for classe, tags in classe_et_edgv_to_tags.items():
        ratio_col = f"step1_consolidado_{classe}_name_ratio"
        inflexao_col = f"step6_consolidado_{classe}_inflexao_data"

        if pd.isna(cell_row.get(ratio_col)) or cell_row[ratio_col] <= 0:
            continue

        data_inicio_str = cell_row.get(inflexao_col) # Uso do inflexao_data como início da janela temporal
        if not isinstance(data_inicio_str, str) or data_inicio_str.strip() == "" or data_inicio_str.lower() == "none":
            continue

        try:
            data_inicio = pd.to_datetime(data_inicio_str, errors="coerce")
            if pd.isna(data_inicio):
                continue
        except Exception as e:
            log_mensagem(id_celula, f"[ERRO] Conversão de data inflexão inválida: {data_inicio_str} - {e}")
            continue

        data_fim = pd.Timestamp("2025-04-06T13:00Z").strftime("%Y-%m-%d") # data_fim fixa de acordo com API metadata ('temporalExtent')
        data_inicio_str = data_inicio.strftime("%Y-%m-%d")

        for tag, value in tags:
            payload = {
                "bboxes": f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
                "time": f"{data_inicio_str},{data_fim}",
                "filter": f"{tag}={value} and name=*",
                "properties": "metadata,tags",
                "clipGeometry": "false"
            }

            try:
                response = requests.post(url_ohsome_latest, data=payload)
                
                print(f"[DEBUG] Célula: {id_celula}, Classe: {classe}, Tag={tag}, Value={value}")
                print(f"[DEBUG] Payload: {json.dumps(payload)}")
                print(f"[DEBUG] Status Code: {response.status_code}")
                print(f"[DEBUG] Response Text: {response.text[:300]}")  # apenas os primeiros 300 chars

                response.raise_for_status()
                dados = response.json()

                for feat in dados.get("features", []):
                    geom_data = feat.get("geometry")
                    props = feat.get("properties", {})

                    if geom_data is None:
                        continue

                    geom = shape(geom_data)
                    if geom.geom_type in ["Polygon", "MultiPolygon"]:
                        geom = geom.centroid

                    props_clean = {
                        "id_celula": id_celula,
                        "classe": classe,
                        "tag": tag,
                        "value": value,
                        **props
                    }

                    features_resultantes.append({
                        "type": "Feature",
                        "geometry": mapping(geom),
                        "properties": props_clean
                    })

            except Exception as e:
                log_mensagem(id_celula, f"[ERRO OHSOME {classe}] {tag}={value}: {str(e)}")

    return features_resultantes

try:
    # === EXECUÇÃO EM LOTE ===
    ultimo_lote = 0
    if ultimo_lote_path.exists():
        with open(ultimo_lote_path, 'r') as f:
            ultimo_lote = int(f.read().strip())

    lote_size = 20
    total_lotes = math.ceil(len(gdf_cells_valid) / lote_size)

    for lote_index in range(ultimo_lote, total_lotes):
        start_time = time.time()
        features_final = []

        subset = gdf_cells_valid.iloc[lote_index * lote_size: (lote_index + 1) * lote_size]

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(processar_ultima_contribuicao, row) for _, row in subset.iterrows()]
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Lote {lote_index + 1} (Step 7)"):
                try:
                    features_final.extend(future.result())
                except Exception as e:
                    log_mensagem(lote_index + 1, f"[FALHA GERAL]: {e}")

        # Salva lote em GeoJSON
        fc = {"type": "FeatureCollection", "features": features_final}
        out_path = output_dir / f"step7_lote{lote_index + 1}.geojson"
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(fc, f)
        log_mensagem(lote_index + 1, f"SALVO {out_path.name}")

        # Atualiza consolidação incremental
        arquivos = sorted(output_dir.glob("step7_lote*.geojson"))
        todas_features = []
        for arquivo in arquivos:
            with open(arquivo, 'r', encoding='utf-8') as f:
                fc_parcial = json.load(f)
                todas_features.extend(fc_parcial['features'])

        final_fc = {"type": "FeatureCollection", "features": todas_features}
        with open(output_dir / "step7_consolidado.geojson", 'w', encoding='utf-8') as f:
            json.dump(final_fc, f)
        log_mensagem(lote_index + 1, "CONSOLIDADO atualizado")

        with open(ultimo_lote_path, 'w') as f:
            f.write(str(lote_index + 1))

        tempo_msg = f"Tempo lote {lote_index + 1}: {str(timedelta(seconds=int(time.time() - start_time)))}"
        print(tempo_msg)
        log_mensagem(lote_index + 1, tempo_msg)

    print("Step 7 (último topônimo por classe) finalizado com sucesso.")
    log_mensagem("step7", "Processamento finalizado")

finally:
    keep_alive_running = False
    keep_alive_thread.join(timeout=1)

Lote 1 (Step 7):   0%|          | 0/20 [00:00<?, ?it/s]

[DEBUG] Célula: 200ME60436N90998, Classe: edif_saude, Tag=amenity, Value=hospital
[DEBUG] Payload: {"bboxes": "-43.97366509596736,-19.87073293787472,-43.9716764017445,-19.86887387732753", "time": "2014-04-08,2025-04-06", "filter": "amenity=hospital and name=*", "properties": "metadata,tags", "clipGeometry": "false"}
[DEBUG] Status Code: 200
[DEBUG] Response Text: {
  "attribution" : {
    "url" : "https://ohsome.org/copyrights",
    "text" : "© OpenStreetMap contributors"
  },
  "apiVersion" : "1.10.4",
  "type" : "FeatureCollection",
  "features" : []
}

[DEBUG] Célula: 200ME60430N90998, Classe: edif_ensino, Tag=amenity, Value=school
[DEBUG] Payload: {"bboxes": "-43.97942463583826,-19.87092593148651,-43.97743597848756,-19.869066905113527", "time": "2021-02-08,2025-04-06", "filter": "amenity=school and name=*", "properties": "metadata,tags", "clipGeometry": "false"}
[DEBUG] Status Code: 200
[DEBUG] Response Text: {
  "attribution" : {
    "url" : "https://ohsome.org/copyrights",
    "t

Lote 1 (Step 7):  10%|█         | 2/20 [02:44<24:41, 82.28s/it]

[DEBUG] Célula: 200ME60430N91000, Classe: edif_ensino, Tag=amenity, Value=kindergarten
[DEBUG] Payload: {"bboxes": "-43.97949343127681,-19.869131222827605,-43.97750478831529,-19.867272204548556", "time": "2018-01-08,2025-04-06", "filter": "amenity=kindergarten and name=*", "properties": "metadata,tags", "clipGeometry": "false"}
[DEBUG] Status Code: 200
[DEBUG] Response Text: {
  "attribution" : {
    "url" : "https://ohsome.org/copyrights",
    "text" : "© OpenStreetMap contributors"
  },
  "apiVersion" : "1.10.4",
  "type" : "FeatureCollection",
  "features" : []
}

[DEBUG] Célula: 200ME60434N90998, Classe: edif_ensino, Tag=amenity, Value=kindergarten
[DEBUG] Payload: {"bboxes": "-43.975584941691636,-19.870797281669184,-43.97359626005931,-19.86893823191383", "time": "2007-10-08,2025-04-06", "filter": "amenity=kindergarten and name=*", "properties": "metadata,tags", "clipGeometry": "false"}
[DEBUG] Status Code: 200
[DEBUG] Response Text: {
  "attribution" : {
    "url" : "https://ohsom

Lote 1 (Step 7):  15%|█▌        | 3/20 [03:20<17:51, 63.01s/it]

...

Lote 1 (Step 7):  95%|█████████▌| 19/20 [40:28<02:58, 178.71s/it]

[DEBUG] Célula: 200ME60434N91004, Classe: edif_constr_lazer, Tag=tourism, Value=museum
[DEBUG] Payload: {"bboxes": "-43.975791405349014,-19.86541318267217,-43.97380276598483,-19.863554157198564", "time": "2015-01-08,2025-04-06", "filter": "tourism=museum and name=*", "properties": "metadata,tags", "clipGeometry": "false"}
[DEBUG] Status Code: 200
[DEBUG] Response Text: {
  "attribution" : {
    "url" : "https://ohsome.org/copyrights",
    "text" : "© OpenStreetMap contributors"
  },
  "apiVersion" : "1.10.4",
  "type" : "FeatureCollection",
  "features" : []
}

[DEBUG] Célula: 200ME60438N91004, Classe: edif_constr_lazer, Tag=building, Value=stadium
[DEBUG] Payload: {"bboxes": "-43.97195179394009,-19.865284487888744,-43.96996313029416,-19.863425439932087", "time": "2012-07-08,2025-04-06", "filter": "building=stadium and name=*", "properties": "metadata,tags", "clipGeometry": "false"}
[DEBUG] Status Code: 200
[DEBUG] Response Text: {
  "attribution" : {
    "url" : "https://ohsome.org/co

Lote 1 (Step 7): 100%|██████████| 20/20 [44:29<00:00, 133.45s/it]

[DEBUG] Célula: 200ME60438N91004, Classe: edif_constr_lazer, Tag=tourism, Value=museum
[DEBUG] Payload: {"bboxes": "-43.97195179394009,-19.865284487888744,-43.96996313029416,-19.863425439932087", "time": "2012-07-08,2025-04-06", "filter": "tourism=museum and name=*", "properties": "metadata,tags", "clipGeometry": "false"}
[DEBUG] Status Code: 200
[DEBUG] Response Text: {
  "attribution" : {
    "url" : "https://ohsome.org/copyrights",
    "text" : "© OpenStreetMap contributors"
  },
  "apiVersion" : "1.10.4",
  "type" : "FeatureCollection",
  "features" : []
}

Tempo lote 1: 0:44:29
Step 7 (último topônimo por classe) finalizado com sucesso.


Ainda trabalhando...


##### Testes

In [ ]:
# teste
import requests

url = "https://api.ohsome.org/v1/contributions/latest/geometry"
params = {
    "bboxes": "-43.9795,-19.8695,-43.9775,-19.8675",
    "time": "2018-02-01,2025-04-06",
    "filter": "amenity=school and name=*",
    "properties": "metadata,tags",
    "clipGeometry": "false"
}

response = requests.post(url, data=params)

print("[DEBUG] Status Code:", response.status_code)
print("[DEBUG] Raw Text:", response.text)

if response.status_code == 200:
    data = response.json()
    print(f"[OK] Features retornadas: {len(data.get('features', []))}")
else:
    print("[ERRO] Algo deu errado na requisição.")


In [ ]:
import requests
URL = 'https://api.ohsome.org/v1/contributions/latest/geometry'
data = {"bboxes": "8.6644,49.4010,8.6663,49.4027", "time": "2014-07-01,2020-06-29", "filter": "landuse=construction and type:way", "showMetadata": "yes", "properties": "metadata,tags", "clipGeometry": "false"}
response = requests.post(URL, data=data)
print(response.json())

##### PostGIS - Close the database connection

In [ ]:
# Close the database connection
if conn and conn.closed == 0:
    # conexão ainda aberta
    with conn.cursor() as cur:
        ...
        cur.close()
    conn.close()
    print("Conexão com o banco de dados fechada.")
else:
    print("Conexão com o banco de dados já estava fechada ou não foi estabelecida.")
